## AI and Radiologist Comparison
Final script for metric extraction and performance evaluation

In [1]:
import os
import fnmatch
import sys
import shutil
import re
import pandas as pd
from sklearn.metrics import *
from scipy import stats
import numpy as np

In [2]:
df_load = "full_df.json"
full_df = pd.read_json(df_load)
full_df.reset_index(drop=True, inplace=True)

## Dataset

In [3]:
print(f" A total of {full_df.shape[0]-1} unique patients and {full_df.shape[0]*4} (MLO and CC) views were evaluted.")

 A total of 839 unique patients and 3360 (MLO and CC) views were evaluted.


## Creating dichotomous variable (Non-dense vs Dense)

In [4]:
#create column for non-dense or dense breast 
def RAD_dense_non_dense(x):
    if x['density'] <3 :
        return 0
    else:
        return 1
    
def AI_dense_non_dense(x):
    if x['aidensity'] <3 :
        return 0
    else:
        return 1

full_df['radiologist_2_densities']=full_df.apply(lambda x : RAD_dense_non_dense(x),axis=1)    
full_df['ai_2_densities']=full_df.apply(lambda x : AI_dense_non_dense(x),axis=1)

## Computing metrics by Radiologist

### Non-dense vs dense category

In [5]:
accuracy2=[]
kappa2=[]
for i in range(5):
    new_df = full_df[full_df['radiologist']==i]
    y_true = new_df["radiologist_2_densities"]
    y_pred = new_df["ai_2_densities"]
    accuracy2.append(accuracy_score(y_true, y_pred))
    kappa2.append(cohen_kappa_score(y_true, y_pred))
    print(f"Using 2 density scale: Annotations from Radiologist {i}: Accuracy is :{accuracy_score(y_true, y_pred)} and cohen kappa is:{cohen_kappa_score(y_true, y_pred)}.")

accuracy2CI = stats.t.interval(alpha=0.95, loc=np.mean(accuracy2), df=len(accuracy2)-1, scale=stats.sem(accuracy2)) 
kappa2CI = stats.t.interval(alpha=0.95, loc=np.mean(kappa2), df=len(kappa2)-1, scale=stats.sem(kappa2)) 


print(f"\n2 Density scale mean accuracy:{np.mean(accuracy2)} and 95% C.I. {accuracy2CI}.") 
print(f"2 Density scale mean cohen kappa:{np.mean(kappa2)} and 95% C.I. {kappa2CI }.")  
print(f"2 Density scale - accuracy standard deviation:{np.std(accuracy2)}.") 

Using 2 density scale: Annotations from Radiologist 0: Accuracy is :0.825938566552901 and cohen kappa is:0.6461268856418878.
Using 2 density scale: Annotations from Radiologist 1: Accuracy is :0.780952380952381 and cohen kappa is:0.5291020766305937.
Using 2 density scale: Annotations from Radiologist 2: Accuracy is :0.8351648351648352 and cohen kappa is:0.644808743169399.
Using 2 density scale: Annotations from Radiologist 3: Accuracy is :0.8958333333333334 and cohen kappa is:0.7923875432525951.
Using 2 density scale: Annotations from Radiologist 4: Accuracy is :0.7066666666666667 and cohen kappa is:0.4099767566601109.

2 Density scale mean accuracy:0.8089111565340235 and 95% C.I. (0.7216179341957255, 0.8962043788723214).
2 Density scale mean cohen kappa:0.6044804010709173 and 95% C.I. (0.426502410170061, 0.7824583919717736).
2 Density scale - accuracy standard deviation:0.06288128814423585.


### Four BI-RADS density categories

In [6]:
accuracy4=[]
kappa4=[]
for i in range(5):
    new_df = full_df[full_df['radiologist']==i]
    y_true_4scale = new_df["density"].values
    y_pred_4scale = new_df["aidensity"].values
    accuracy4.append(accuracy_score(y_true_4scale, y_pred_4scale))
    kappa4.append(cohen_kappa_score(y_true_4scale, y_pred_4scale))
    print(f"Using 4 density scale: Annotations from Radiologist {i}:Accuracy is :{accuracy_score(y_true_4scale, y_pred_4scale)} and cohen kappa is:{cohen_kappa_score(y_true_4scale, y_pred_4scale)}.")

accuracy4CI = stats.t.interval(alpha=0.95, loc=np.mean(accuracy4), df=len(accuracy4)-1, scale=stats.sem(accuracy4)) 
kappa4CI = stats.t.interval(alpha=0.95, loc=np.mean(kappa4), df=len(kappa4)-1, scale=stats.sem(kappa4)) 
    
print(f"\n4 Density scale mean accuracy:{np.mean(accuracy4)}, and 95% C.I. {accuracy4CI }.") 
print(f"4 Density scale mean cohen kappa:{np.mean(kappa4)} and 95% C.I. {kappa4CI }.")  
print(f"4 Density scale - accuracy standard deviation:{np.std(accuracy4)}.") 


Using 4 density scale: Annotations from Radiologist 0:Accuracy is :0.6450511945392492 and cohen kappa is:0.41948143491265166.
Using 4 density scale: Annotations from Radiologist 1:Accuracy is :0.6333333333333333 and cohen kappa is:0.3565715649994031.
Using 4 density scale: Annotations from Radiologist 2:Accuracy is :0.6483516483516484 and cohen kappa is:0.4021761445288442.
Using 4 density scale: Annotations from Radiologist 3:Accuracy is :0.4791666666666667 and cohen kappa is:0.2882562277580071.
Using 4 density scale: Annotations from Radiologist 4:Accuracy is :0.43333333333333335 and cohen kappa is:0.1574704288640718.

4 Density scale mean accuracy:0.567847235244846, and 95% C.I. (0.4395765622046476, 0.6961179082850445).
4 Density scale mean cohen kappa:0.3247911602125956 and 95% C.I. (0.1926608020484431, 0.45692151837674805).
4 Density scale - accuracy standard deviation:0.0923992142326619.
